In [ ]:
import pandas as pd
import historic
from pathlib import Path
import cityRemap
from sklearn.utils import resample
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt

## import testset

In [ ]:
testset = pd.read_csv(Path('extracted/high/AMS.csv'))
testset['dt'] = pd.DatetimeIndex(testset.dt)
testset = testset[testset['dt'].dt.year==2020]
testset = testset[['dt','Temperature']]

## get date range for tbats

In [ ]:
df = pd.DataFrame(
        {'dt': pd.date_range('2021-01-01', '2022-01-01', freq='1H', closed='left')})

In [ ]:
date = pd.DataFrame()
for month in range(1,13):
    month_data = df[df.dt.dt.month==month]
    date = date.append(month_data.iloc[0:24],ignore_index=True)
date = pd.DatetimeIndex(date.dt)

## get histrical interval

In [ ]:
def get_historical(conf,year):    
    historic_interval = pd.DataFrame()
    historic_data = pd.read_csv('extracted/high/AMS.csv')
    historic_data.dt = pd.DatetimeIndex(historic_data.dt)
    historic_data = historic_data[(historic_data.dt.dt.year>2019-year) & (historic_data.dt.dt.year<2020)]
    for month in range(1,13):
        for hour in range(24):  
                points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
                lower = points['Temperature'].quantile((1-conf)/2)
                higher = points['Temperature'].quantile(1-((1-conf)/2))
                historic_interval = historic_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
    historic_interval = historic_interval.set_index(date)
    return historic_interval

In [ ]:
historic_data = pd.read_csv('extracted/high/AMS.csv')
temp  =historic_data[['dt','Temperature']]
temp.dt = pd.DatetimeIndex(temp.dt)

In [ ]:
def result(interval_set):
    correct = 0
    total = 0
    old_correct = 0
    for hour in range(24):
        for month in range(1,13):
            upper,lower = interval_set[(interval_set.index.hour==hour) & (interval_set.index.month==month)].iloc[0].values
            points = testset.loc[(testset.dt.dt.hour==hour) & (testset.dt.dt.month==month)]
            total += len(points)
            correct += points['Temperature'].between(lower,upper).sum()
        old_correct = correct
    return correct/total

In [ ]:
def get_optimal(conf):
    optimal = pd.DataFrame()
    for month in range(1,13):
        for hour in range(24):
            points = testset[(testset.dt.dt.month==month) & (testset.dt.dt.hour==hour)]
            upper = points['Temperature'].quantile(conf)
            lower = points['Temperature'].quantile(1-conf)
            optimal = optimal.append({'Lower':lower,'Higher':upper},ignore_index=True)
    return optimal

In [ ]:
def get_width(interval):
    return (interval['Higher']-interval['Lower']).mean()

In [ ]:
conf = [0.8,0.9,0.95,0.99,1]

In [ ]:
optimal_line = pd.DataFrame()
for con in conf:
    optimal_line = optimal_line.append({'Confidence':con,'Width':get_width(get_optimal(con))},ignore_index=True)

In [ ]:
hist_points = pd.DataFrame()
for con in conf:
    intervals = get_historical(con,10)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'Historical interval 10y'}, ignore_index=True)
for con in conf:
    intervals = get_historical(con,5)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'Historical interval 5y'}, ignore_index=True)
for con in conf:
    intervals = get_historical(con,3)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'Historical interval 3y'}, ignore_index=True)
for con in conf:
    intervals = get_historical(con,1)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'Historical interval 1y'}, ignore_index=True)    

In [ ]:
fg = sns.FacetGrid(data=hist_points, palette=sns.color_palette(),hue='Method', size=5, aspect=1.5)
fg.map(plt.scatter,'Confidence', 'Width').add_legend()
fg.axes[0,0].plot(optimal_line.Confidence, optimal_line['Width'], marker="o", label='Optimal')
plt.title("Amsterdam 2020")
plt.xlim(0.50, 1.01)
plt.show()

# TBATS

In [ ]:
inter = get_historical(0.95,3)

In [ ]:
test = pd.read_excel('AMStbatsAVG.xlsx')
for con in conf:
    sarimax_interval = pd.DataFrame()
    intervals = get_historical(0.95,10)
    for i in range(len(test)):
        higher = intervals.iloc[i]['Higher']
        lower = intervals.iloc[i]['Lower']
        half = (higher-lower)/2
        upper = test.iloc[i]['pointForcast']+half
        lower = test.iloc[i]['pointForcast']-half
        sarimax_interval = sarimax_interval.append({'Lower':lower,'Higher':upper},ignore_index=True)
    sarimax_interval = sarimax_interval.set_index(date)

    hist_points = hist_points.append({'Type':'TBATS','Confidence':result(sarimax_interval),'Width':get_width(sarimax_interval),'Method':'TBATS'}, ignore_index=True)

## get tbats interval

In [ ]:
# ams_tbats = pd.read_excel('AMSexportTbatsConf95final.xlsx')
# ams_tbats = ams_tbats.rename(columns={'lo':'Lower','hi':'Higher'})
# tbats_interval = pd.DataFrame()
# for month in range(12):
#         for hour in range(24):
#             points = ams_tbats.loc[(ams_tbats.index.hour==hour) & (ams_tbats.index.month==month+1)]
#             lower = points['Temperature'].quantile(0)
#             higher = points['Temperature'].quantile(1)
#             tbats_interval = tbats_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
# tbats_interval = tbats_interval.set_index(date.dt)

# LSTM mean

In [ ]:
mean_ams = pd.read_csv('LSTM_mean_AMS.csv')
width_ams = pd.read_csv('ams_pred_width.csv')

In [ ]:
ams_mean_interval = pd.DataFrame()
for i in range(len(mean_ams['Mean'])):
    half = abs(width_ams['Width'][i]/2)
    point = mean_ams['Mean'][i]
    upper_point = point + half
    lower_point = point - half
    ams_mean_interval = ams_mean_interval.append({'Lower':lower_point,'Higher':upper_point},ignore_index=True)
ams_mean_interval.index =date
intervalswidth = get_width(ams_mean_interval)
true_confidence = result(ams_mean_interval)
hist_points = hist_points.append({'Type':'LSTM both','Confidence':true_confidence,'Width':intervalswidth,'Method':'LSTM both'}, ignore_index=True)

In [ ]:
hist_points

In [ ]:
for con in conf:
    ams_mean_interval = pd.DataFrame()
    intervals = get_historical(con,10)
    for i in range(len(mean_ams['0'])):
        higher = intervals.iloc[i]['Higher']
        lower = intervals.iloc[i]['Lower']
        half = (higher-lower)/2
        point = mean_ams.iloc[i]['0']
        upper_point = point + half
        lower_point = point - half
        ams_mean_interval = ams_mean_interval.append({'Lower':lower_point,'Higher':upper_point},ignore_index=True)
    ams_mean_interval.index =date
    intervalswidth = get_width(ams_mean_interval)
    true_confidence = result(ams_mean_interval)
    hist_points = hist_points.append({'Type':'LSTM','Confidence':true_confidence,'Width':intervalswidth,'Method':'LSTM'}, ignore_index=True)

# Non parametric bootstrap

In [ ]:
# def get_historical_boot(conf,year):    
#     historic_interval = pd.DataFrame()
#     historic_data = pd.read_csv('extracted/high/AMS.csv')
#     historic_data.dt = pd.DatetimeIndex(historic_data.dt)
#     historic_data = historic_data[(historic_data.dt.dt.year>2019-year) & (historic_data.dt.dt.year<2020)]
#     for month in range(1,13):
#         for hour in range(24):
#                 points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
#                 points = resample(points, replace=True, n_samples=50000, random_state=1)
#                 lower = points['Temperature'].quantile(1-conf)
#                 higher = points['Temperature'].quantile(conf)
#                 historic_interval = historic_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
#     historic_interval = historic_interval.set_index(date)
#     return historic_interval

In [ ]:
def get_historical_boot(conf,year):    
    historic_interval = pd.DataFrame()
    historic_data = pd.read_csv('extracted/high/AMS.csv')
    historic_data.dt = pd.DatetimeIndex(historic_data.dt)
    historic_data = historic_data[(historic_data.dt.dt.year>2019-year) & (historic_data.dt.dt.year<2020)]
    for month in range(1,13):
        for hour in range(24):
                points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
                points_size = len(points)
                higher_b,lower_b = [],[]
                for i in range(1000):
                    points = resample(points, replace=True, n_samples=points_size, random_state=1)
                    lower_b.append(points['Temperature'].quantile(1-conf))
                    higher_b.append(points['Temperature'].quantile(conf))
                historic_interval = historic_interval.append({'Lower':np.mean(lower_b),'Higher':np.mean(higher_b)}, ignore_index=True)
    historic_interval = historic_interval.set_index(date)
    return historic_interval

In [ ]:
intervals = get_historical_boot(0.95,5)
width = get_width(intervals)
true_confidence = result(intervals)
hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'5y bootstrap'}, ignore_index=True)    

In [ ]:
def get_historical_boot(conf,year):    
    historic_interval = pd.DataFrame()
    historic_data = pd.read_csv('extracted/high/AMS.csv')
    historic_data.dt = pd.DatetimeIndex(historic_data.dt)
    historic_data = historic_data[(historic_data.dt.dt.year>2019-year) & (historic_data.dt.dt.year<2020)]
    for month in range(1,13):
        for hour in range(24):
                points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
                points = resample(points, replace=True, n_samples=50000, random_state=1)
                lower = points['Temperature'].quantile(1-conf)
                higher = points['Temperature'].quantile(conf)
                historic_interval = historic_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
    historic_interval = historic_interval.set_index(date)
    return historic_interval

In [ ]:
historic_data = pd.read_csv('extracted/high/AMS.csv')
historic_data.dt = pd.DatetimeIndex(historic_data.dt)
historic_data = historic_data[(historic_data.dt.dt.year>2019-5) & (historic_data.dt.dt.year<2020)]

points = historic_data.loc[(historic_data.dt.dt.hour==1) & (historic_data.dt.dt.month==1)]
points = resample(points, replace=True, n_samples=10000, random_state=1)

In [ ]:
for con in conf:
    intervals = get_historical_boot(con,5)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'5y bootstrap'}, ignore_index=True)    

In [ ]:
for con in conf:
    intervals = get_historical_boot(con,10)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'10y bootstrap'}, ignore_index=True) 

# Parametric bootstrap

In [ ]:
def get_historical_par_boot(conf,year):    
    historic_interval = pd.DataFrame()
    historic_data = pd.read_csv('extracted/high/AMS.csv')
    historic_data.dt = pd.DatetimeIndex(historic_data.dt)
    historic_data = historic_data[(historic_data.dt.dt.year>2019-year) & (historic_data.dt.dt.year<2020)]
    for month in range(1,13):
        for hour in range(24):
                points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
                num_points = len(points)
                upper,lower = [],[]
                for i in range(1000):
                    np.random.normal(points.mean(),points.std(), num_points)
                    points = resample(points, replace=True, n_samples=num_points, random_state=1)
                    upper.append(points.quantile(conf))
                    lower.append(points.quantile(1-conf))
                historic_interval = historic_interval.append({'Lower':np.mean(lower),'Higher':np.mean(higher)}, ignore_index=True)
    historic_interval = historic_interval.set_index(date)
    return historic_interval

In [ ]:
for con in conf:
    intervals = get_historical_boot(con,5)
    width = get_width(intervals)
    true_confidence = result(intervals)
    hist_points = hist_points.append({'Type':'Hist {}'.format(con),'Confidence':true_confidence,'Width':width,'Method':'5y param bootstrap'}, ignore_index=True) 

In [ ]:
fg = sns.FacetGrid(data=hist_points, palette=sns.color_palette(),hue='Method', size=5, aspect=1.5)
fg.map(plt.scatter,'Confidence', 'Width').add_legend()
fg.axes[0,0].plot(optimal_line.Confidence, optimal_line['Width'], marker="o", label='Optimal')
plt.title("Amsterdam 2020")
plt.xlim(0.50, 1.01)
plt.show()

In [ ]:
for con in conf:
    tbats_interval = pd.DataFrame()
    intervals = get_historical(con,5)
    for i in range(len(allpoint)):
        half = get_width(intervals.iloc[i]/2)
        upper = allpoint.iloc[i]['pointForecast']+half
        lower = allpoint.iloc[i]['pointForecast']-half
        tbats_interval = tbats_interval.append({'Lower':lower,'Higher':upper},ignore_index=True)
    tbats_interval = tbats_interval.set_index(date)

    hist_points = hist_points.append({'Type':'allpoint','Confidence':result(sarimax_interval),'Width':get_width(sarimax_interval),'Method':'allpoint'}, ignore_index=True)

In [ ]:
allpoints_mean = pd.DataFrame()

for month in range(1,13):
    for hour in range(24):  
            points = allpoint.loc[(allpoint.index.hour==hour) & (allpoint.index.month==month)]
            allpoints_mean = allpoints_mean.append(points.mean(),ignore_index=True)

In [ ]:
for con in conf:
    sarimax_interval = pd.DataFrame()
    intervals = get_historical(con,5)
    width = get_width(intervals)
    half = width/2
    for i in range(len(allpoints_mean)):
        upper = allpoints_mean.iloc[i]['pointForcast']+half
        lower = allpoints_mean.iloc[i]['pointForcast']-half
        sarimax_interval = sarimax_interval.append({'Lower':lower,'Higher':upper},ignore_index=True)
    sarimax_interval = sarimax_interval.set_index(date)

    hist_points = hist_points.append({'Type':'all point','Confidence':result(sarimax_interval)*100,'Width':get_width(sarimax_interval),'Method':'all point'}, ignore_index=True)

In [ ]:
hist_points

In [ ]:
get_width(allpoint_int)

In [ ]:
test

In [ ]:
tbats_interval = pd.read_excel('AMSexportTbatsHiLoAndErrorfinal.xlsx')

In [ ]:
tbats_interval = tbats_interval.rename(columns={'lo':'Lower','hi':'Higher'})

In [ ]:
interval_set = tbats_interval
correct = 0
total = 0
old_correct = 0
for hour in range(24):
    for month in range(1,13):
        lower,upper = interval_set[(interval_set.index.hour==hour) & (interval_set.index.month==month)].iloc[0].values
        points = testset.loc[(testset.dt.dt.hour==hour) & (testset.dt.dt.month==month)]
        total += len(points)
        correct += points['Temperature'].between(lower,upper).sum()
    old_correct = correct
print(correct/total)

In [ ]:
get_width(tbats_interval)

In [ ]:
tbats_interval = tbats_interval.set_index(date)

In [ ]:
result(tbats_interval)

In [ ]:
tbats_interval

In [ ]:
tbats_predict = pd.read_excel('AMS_TBATS.xlsx')

In [ ]:
for con in conf:
    tbats_interval = pd.DataFrame()
    intervals = get_historical(con,5)
    for i in range(len(tbats_predict)):
        half = get_width(intervals.iloc[i]/2)
        upper = tbats_predict.iloc[i]['pointForecast']+half
        lower = tbats_predict.iloc[i]['pointForecast']-half
        tbats_interval = tbats_interval.append({'Lower':lower,'Higher':upper},ignore_index=True)
    tbats_interval = tbats_interval.set_index(date)

    hist_points = hist_points.append({'Type':'TBATS 5y width','Confidence':result(sarimax_interval),'Width':get_width(sarimax_interval),'Method':'TBATS 5y'}, ignore_index=True)

In [ ]:
test = test.set_index(date)

In [ ]:
hist_points = hist_points.append({'Type':'Combi','Confidence':result(test),'Width':get_width(test),'Method':'Combi'}, ignore_index=True)

In [ ]:
hist_points

In [ ]:
fg = sns.FacetGrid(data=hist_points, palette=sns.color_palette(),hue='Method', size=5, aspect=1.5)
fg.map(plt.scatter,'Confidence', 'Width').add_legend()
fg.axes[0,0].plot(optimal_line.Confidence, optimal_line['Width'], marker="o", label='Optimal')
plt.title("Amsterdam 2020")
plt.xlim(0.50, 1.01)
plt.show()

In [ ]:
historic_interval = get_historical(con,5)

In [ ]:
fg = sns.FacetGrid(data=hist_points, palette=sns.color_palette(),hue='Method', size=5, aspect=1.5)
fg.map(plt.scatter, 'Confidence', 'Width').add_legend()
fg.axes[0,0].plot(final.Confidence, final['Width'], marker="o", label='Optimal')
plt.title("Amsterdam 2020")
plt.xlim(50, 101)
plt.show()

In [ ]:
tbats = pd.read_excel('AMS_TBATS.xlsx')

In [ ]:
tbats = tbats.set_index(testset.index)

In [ ]:
testset = testset[testset['dt'].dt.year==2020]

In [ ]:
testset_mean = pd.DataFrame()
for month in range(1,13):
    for hour in range(24):  
            points = testset.loc[(testset.dt.dt.hour==hour) & (testset.dt.dt.month==month)]
            testset_mean = testset_mean.append(points.mean(numeric_only=True), ignore_index=True)
testset_mean = testset_mean.set_index(date)

In [ ]:
tbats = tbats.set_index(date)

In [ ]:
testset_mean = testset_mean.reset_index()
tbats = tbats.reset_index()
ax = testset_mean['Temperature'].plot(legend=True, figsize=(16,8))
tbats['pointForecast'].plot(legend=True, title='SARIMAX prediction', ylabel='Temperature')

In [ ]:
ax = testset_mean['Temperature'].plot(legend=True, figsize=(16,8))
confidence_prediction['Temperature'].plot(legend=True, title='SARIMAX prediction', ylabel='Temperature')

In [ ]:
conf = get_historical(0.95,5)

In [ ]:
confidence_prediction = pd.DataFrame({'Temperature':conf['Higher']-conf['Lower']})

In [ ]:
confidence_prediction = confidence_prediction.reset_index()

In [ ]:
point = point.reset_index()

In [ ]:
point = point.values

In [ ]:
combined = pd.DataFrame({'TBATS':tbats['pointForecast'],'Temperature':testset_mean['Temperature'],'Historical':point})

In [ ]:
combined

In [ ]:
import plotly.express as px

fig = px.line(combined, y=['TBATS','Temperature','Historical'], x=testset_mean.index,labels={'value':"Temperature"}, title='Historic interval')

fig.show()

In [ ]:
temp

In [ ]:
combined_interval = pd.DataFrame({'Higher TBATS':tbats_interval['Higher'],'Lower TBATS':tbats_interval['Lower'],'Higher Historic':historic_interval['Higher'],'Lower Historic':historic_interval['Lower'],'Actual Temperature':testset_mean['Temperature']})

In [ ]:
combined_interval.columns

In [ ]:
combined_interval

In [ ]:
combined_interval = combined_interval.drop(columns=['index'])

In [ ]:
import plotly.express as px

fig = px.line(combined_interval, y=['Higher TBATS',	'Lower TBATS','Higher Historic','Lower Historic','Actual Temperature'], x=combined_interval.index,labels={'value':"Temperature"}, title='Historic interval')

fig.show()